In [1]:
### Import Packages ###
import os
import numpy as np
import math as math
import pandas as pd
import random as random
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

### Local Packages ###
from utils.Main import *
from utils.Selector import *
from utils.Auxiliary import *
from utils.Prediction import *

In [2]:
### Get Directory ###
cwd = os.getcwd()
ParentDirectory = os.path.abspath(os.path.join(cwd, ".."))

### DataType ###
DataTypeInput = "COMPAS"

# One Iteration

## Inputs

In [3]:
i=1
DataFileInput = "COMPAS"
TopCModels = 50
Seed = 1
TestProportion = 0.2
CandidateProportion = 0.8
ModelType = TreeFarmsFunction
DataArgs = {}
ModelArgs = {"TopCModels": TopCModels, "config" : {"regularization": 0.01, "rashomon_bound_multiplier": 0.03}, "Type":"Classification"}


## One Iteration Function

### Set Up

In [4]:
### Run Time ###
StartTime = time.time()

### Set Up ###
random.seed(Seed)
np.random.seed(Seed)
ErrorVec = []
SelectedObservationHistory = []

### Load Data

In [5]:
from utils.Main import DataGeneratingProcess                             ### NOTE: Why is this not imported from utils.Main import *
### Generate Data ###
if(DataFileInput == "Simulate"):
    df = DataGeneratingProcess(**DataArgs)
else:
    df = LoadData(DataFileInput)

### Train Test Split

In [6]:
### Train Test Candidate Split
from utils.Main import TrainTestCandidateSplit                           ### NOTE: Why is this not imported from utils.Main import *
df_Train, df_Test, df_Candidate = TrainTestCandidateSplit(df, TestProportion, CandidateProportion)

### Argument Processing

# Learning Process Function

In [7]:
### Set Up ###
ErrorVec = []
SelectedObservationHistory = []


In [8]:
i=1

In [9]:
# SelectorArgs = {"Model" : Model, "df_Candidate" : df_Candidate, "df_Train" : df_Train, "TopCModels" : TopCModels},

# ### Selector Arguments ###
# SelectorArgs["df_Train"] = df_Test
# SelectorArgs["df_Candidate"] = df_Candidate
# SelectorArgs["Model"] = ModelType
# SelectorArgsFiltered = FilterArguments(SelectorType, SelectorArgs)


In [10]:
### Model Arguments ###
ModelArgs['df_Train'] = df_Train
ModelArgsFiltered = FilterArguments(ModelType, ModelArgs)

In [11]:
import sys
import pandas as pd
import numpy as np
import pathlib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from treefarms.model.threshold_guess import compute_thresholds, cut
from treefarms import TREEFARMS
from treefarms.model.model_set import ModelSetContainer
sys.path.append('/Users/simondn/Documents/RashomonActiveLearning/Code')
# from treeFarms.treefarms.model.threshold_guess import compute_thresholds

In [12]:
# Summary: Initializes and fits a treefarms model.
# Input:
#   df_Train: The training data.
#   TopCModels: TopCModels top models
#   config:
# Output:
# treeFarmsModel: A treefarms model.

### Libraries ###
from treeFarms.treefarms import TREEFARMS

# import sys
# sys.path.append('/Users/simondn/Documents/RashomonActiveLearning/Code')
# from treeFarms.treefarms.model.threshold_guess import compute_thresholds
# from utils.Prediction.TreeFARMS import <function_or_class_name>

### Function ###
def TreeFarmsFunction(df_Train, config, TopCModels):
   ### Train TreeFarms Model ###
    TreeFarmsModel = TREEFARMS("config")
    TreeFarmsModel.fit(df_Train.loc[:, df_Train.columns != "Y"], df_Train["Y"])
    
    ### Return ###
    return TreeFarmsModel


# NOTE: Is there a way to prune the tree such that only the top models are given back? Look into this


In [13]:
# TreeFarmsModel = TREEFARMS("config")
# TreeFarmsModel.fit(df_Train.loc[:, df_Train.columns != "Y"], df_Train["Y"])

In [14]:
# TreeFarmsFunction(df_Train = df_Train, config = ModelArgs["config"], TopCModels= ModelArgs["TopCModels"])

In [15]:
# TreeFarmsFunction(**ModelArgsFiltered)

In [16]:
# ### Prediction Model ###
# Model = ModelType(**ModelArgsFiltered)
# if "Model" in SelectorArgs.keys(): SelectorArgs['Model'] = Model            # NOTE: THIS IS NOT DYNAMIC

In [17]:
config = {"regularization": 0.01, "rashomon_bound_multiplier": 0.03}
Model = TREEFARMS(config)
Model.fit(df_Train.loc[:, df_Train.columns != "Y"], df_Train["Y"])


null
Finding Optimal Objective...
treefarms reported successful execution{
  "false": {
    "false": {
      "complexity": 0.009999999776482582,
      "loss": 0.1556561142206192,
      "name": "Y",
      "prediction": 0
    },
    "feature": 2,
    "name": "age:<23",
    "reference": 1,
    "relation": "==",
    "true": {
      "complexity": 0.009999999776482582,
      "loss": 0.07963801175355911,
      "name": "Y",
      "prediction": 1
    },
    "type": "integral"
  },
  "feature": 11,
  "model_objective": 0.375701367855072,
  "name": "priors:>3",
  "reference": 1,
  "relation": "==",
  "true": {
    "complexity": 0.009999999776482582,
    "loss": 0.11040724068880081,
    "name": "Y",
    "prediction": 1
  },
  "type": "integral"
}
{
  "false": {
    "false": {
      "complexity": 0.009999999776482582,
      "loss": 0.1556561142206192,
      "name": "Y",
      "prediction": 0
    },
    "feature": 11,
    "name": "priors:>3",
    "reference": 1,
    "relation": "==",
    "true": {
 

In [18]:
### Current Error ###
TestErrorVal = TestErrorFunction(Model, df_Test, ModelArgs)
if(len(TestErrorVal) > 1):
    AllErrors = TestErrorVal                               # Rashomon gives all errors of Rashomon
    CurrentError = float(np.min(AllErrors))                       # Extract the best one
else: 
    CurrentError = TestErrorVal                            # One output for non-Rashomon
ErrorVec.append(CurrentError)

In [19]:
# RashomonQBCSelector #
def RashomonQBCFunction(Model, df_Candidate, df_Train, TopCModels, AllErrors):

    # ### GSx ### Incorporate? Good for tie breakers.
    # d_nmX = cdist(df_Candidate.loc[:,df_Candidate.columns!= "Y"], df_Train.loc[:,df_Train.columns!= "Y"], metric = distance)
    # d_nX = d_nmX.min(axis=1)

    ### Extract Errors ###
    LowestErrorIndices = np.argsort(AllErrors)[:TopCModels]

    ### Prediction ###
    PredictedValues = [Model[i].predict(df_Candidate) for i in LowestErrorIndices]
    RashomonMean = np.array(PredictedValues).mean(axis =0)

    ### Uncertainty Metric ###
    df_Candidate["UncertaintyMetric"] = abs(RashomonMean - 0.5)
    IndexRecommendation = int(df_Candidate.sort_values(by = "UncertaintyMetric", ascending = True).index[0])

    return(IndexRecommendation)



In [20]:
### Sampling Procedure ###
QueryObservationIndex = SelectorType(**SelectorArgs)
QueryObservation = df_Candidate.loc[[QueryObservationIndex]]                  # or should this be iloc nah I think its loc lol
SelectedObservationHistory.append(QueryObservationIndex)

### Update Train and Candidate Sets ###
df_Train = pd.concat([df_Train, QueryObservation])
df_Candidate = df_Candidate.drop(QueryObservationIndex)

### Update SelectorArgs and ModelArgs ###                                     # NOTE: THIS IS NOT DYNAMIC
if "df_Train" in ModelArgs.keys(): ModelArgs['df_Train'] = df_Train
if "df_Train" in SelectorArgs.keys(): SelectorArgs['df_Train'] = df_Train
if "df_Candidate" in SelectorArgs.keys(): SelectorArgs['df_Candidate'] = df_Candidate      

NameError: name 'SelectorType' is not defined